# Investigating if split portions have to cleansed before skeletonized

In [59]:
import datajoint as dj
import numpy as np
from meshparty import trimesh_io
import time
from collections import Counter
from tqdm import tnrange, tqdm_notebook
import pymeshfix

import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

In [36]:
len("")

0

In [40]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_NEURITES"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    filename = "neuron_" + str(neuron_ID)
    #get the current file location
    if len(folder) > 0:
        file_loc = pathlib.Path.cwd() / folder
        path_and_filename = file_loc / filename
    else:
        path_and_filename = pathlib.Path.cwd() / filename
        file_loc = pathlib.Path.cwd()
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [60]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [61]:
def meshlab_fix_manifold(path_and_filename):
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [32]:
neuron_id = 648518346349507130
key = dict(segmentation=2,segment_id=neuron)

ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

In [4]:
mesh_neuron = (ta3p100.Mesh() & key).fetch1()

#import the mesh into t trimesh_io object
start_time = time.time()
mesh = trimesh_io.Mesh(vertices=mesh_neuron["vertices"], faces=mesh_neuron["triangles"])

#get the body count
mesh.body_count

21

In [51]:
total_meshes = mesh.split(only_watertight=False)
print(len(total_meshes))

#export the cleansed mesh
write_Whole_Neuron_Off_file("split_mesh",total_meshes[0].vertices,total_meshes[0].faces,folder="")
total_meshes[0].faces.shape

#get the original otuput
write_Whole_Neuron_Off_file(str(neuron_id) + "_orig",mesh.vertices,mesh.faces,folder="")

21
Done writing OFF file


(20684, 3)

In [62]:
#do pymeshfix on both the original and the cleansed one
#print("Step 2: Remove all error semgents")


"""##################  FOR ORIGINAL MESH ##################"""
start_time = time.time()
#ORIGINAL pass the vertices and faces to pymeshfix to become watertight
meshfix_orig = pymeshfix.MeshFix(mesh.vertices,mesh.faces)
meshfix_orig.repair(verbose=False,joincomp=True,remove_smallest_components=False)
print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")

#print("Step 2: Writing Off File")
start_time = time.time()
#write the new mesh to off file
path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(neuron_id) + "_orig",
                                                                  meshfix_orig.v,
                                                                  meshfix_orig.f,
                                                                 folder="")
print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")



"""##################  FOR CLEANSED MESH ##################"""
start_time = time.time()
#ORIGINAL pass the vertices and faces to pymeshfix to become watertight
meshfix_cleansed = pymeshfix.MeshFix(total_meshes[0].vertices,total_meshes[0].faces)
meshfix_cleansed.repair(verbose=False,joincomp=True,remove_smallest_components=False)
print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")

#print("Step 2: Writing Off File")
start_time = time.time()
#write the new mesh to off file
path_and_filename_cleansed,filename_cleansed,file_loc_cleansed = write_Whole_Neuron_Off_file(
                                                                str(neuron_id) + "_cleansed",
                                                                  meshfix_cleansed.v,
                                                                  meshfix_cleansed.f,
                                                                 folder="")
print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")



Step 2: Pymesh shrinkwrapping: 2.908954381942749
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.20247530937194824
Step 2: Pymesh shrinkwrapping: 0.7408139705657959
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.11665463447570801


In [64]:
#fix any non-manifold issues:
print(path_and_filename)
print(path_and_filename_cleansed)

#Run the meshlabserver scripts
start_time = time.time()
output_mesh = meshlab_fix_manifold(path_and_filename)
print(f"Step 4: Meshlab fixing non-manifolds: {time.time() - start_time}")

#Run the meshlabserver scripts
start_time = time.time()
output_mesh = meshlab_fix_manifold(path_and_filename_cleansed)
print(f"Step 4: Meshlab fixing non-manifolds: {time.time() - start_time}")

/notebooks/13_Skeletons/test_error/neuron_648518346349507130_orig
/notebooks/13_Skeletons/test_error/neuron_648518346349507130_cleansed
starting meshlabserver fixing non-manifolds
Step 4: Meshlab fixing non-manifolds: 0.7781343460083008
starting meshlabserver fixing non-manifolds
Step 4: Meshlab fixing non-manifolds: 0.8029096126556396


In [70]:
#how to get the splitting data
mesh_neuron = (ta3p100.Mesh() & key).fetch1()

#import the mesh into t trimesh_io object
start_time = time.time()
mesh = trimesh_io.Mesh(vertices=mesh_neuron["vertices"], faces=mesh_neuron["triangles"])

count, labels = trimesh_io.trimesh.graph.csgraph.connected_components(
    mesh.edges_sparse,
    directed=False,
    return_labels=True)

print("n_edges = " + str(count))

#using numpy which is faster!!!
start_time = time.time()
values = np.array(labels)
searchval = 0
ii = np.where(values == searchval)[0]
print("percentage = " + str(len(ii)/len(labels)))
print(f"Numpy took {time.time()-start_time} sec ")

#try using the Counter method
start_time = time.time()
myCounter = Counter(labels)
dict(myCounter)[0]
print("percentage = " + str(dict(myCounter)[0]/len(labels)))
print(f"Counter took {time.time()-start_time} sec ")

#get the number of labels equal to 0, the largest group

n_edges = 21
percentage = 0.40413943355119825
Numpy took 0.0014736652374267578 sec 
percentage = 0.40413943355119825
Counter took 0.01138925552368164 sec 


In [ ]:
myCounter = Counter(labels)

In [23]:
"""Did not end up working where used the labels"""
##trying to use submesh function of trimesh
# '''
#     def submesh(self, faces_sequence, **kwargs):
#         """
#         Return a subset of the mesh.
#         Parameters
#         ----------
#         faces_sequence : sequence (m,) int
#           Face indices of mesh
#         only_watertight : bool
#           Only return submeshes which are watertight
#         append : bool
#           Return a single mesh which has the faces appended.
#           if this flag is set, only_watertight is ignored
#         Returns
#         ---------
#         if append : trimesh.Trimesh object
#         else :      list of trimesh.Trimesh objects
#         """
#         return util.submesh(mesh=self,
#                             faces_sequence=faces_sequence,
#                             **kwargs)
# '''
# count, labels = trimesh_io.trimesh.graph.csgraph.connected_components(
#     mesh.edges_sparse,
#     directed=False,
#     return_labels=True)
# mesh._cache['vertices_component_label'] = labels
# #count
# #labels

# myCounter = Counter(labels)
# myCounter 


# import time
# start_time = time.time()
# faces_sequence = [i for i,x in enumerate(labels) if x == 0]
# print(f"Basic array took {time.time()-start_time} sec ")
# #print(faces_sequence)

# start_time = time.time()
# values = np.array(labels)
# searchval = 0
# ii = np.where(values == searchval)[0]
# print(f"Numpy took {time.time()-start_time} sec ")
# print(type(ii.tolist()))
# #conclusion: Numpy was a lot faster than manual method

# #try to get the submesh
# filtered_mesh = mesh.submesh(faces_sequence = [ii.tolist()],append=True)

# dir(filtered_mesh)
# print(mesh.faces.shape)
# print(mesh.vertices.shape)

# print(filtered_mesh.faces.shape)
# print(filtered_mesh.vertices.shape)


# #output the cleansed mesh
# write_Whole_Neuron_Off_file(str(neuron_id) + "_cleansed",filtered_mesh.vertices,filtered_mesh.faces,folder="")


Basic array took 0.0842580795288086 sec 
Numpy took 0.0008189678192138672 sec 
<class 'list'>
